In [2]:
import numpy as np
import pandas as pd
import nltk
import sklearn as sk
import re # regex
import matplotlib

import tensorflow as tf
from tensorflow import keras
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
#load datasets
news_df = pd.read_csv('../datasets/news-20k-comments.csv')
politics_df = pd.read_csv("../datasets/politics-20k-comments.csv")
politicaldiscussion_df = pd.read_csv('../datasets/politicaldiscussion-20k-comments.csv')

news_df = news_df.filter(['created_utc', 'body'])
politics_df = politics_df.filter(['created_utc', 'body'])
politicaldiscussion_df = politicaldiscussion_df.filter(['created_utc', 'body'])

In [4]:
#filter out deleted comments
news_df = news_df[news_df['body'] != "[deleted]"]
politics_df = politics_df[politics_df['body'] != "[deleted]"]
politicaldiscussion_df = politicaldiscussion_df[politicaldiscussion_df['body'] != "[deleted]"]

In [5]:
# clean out any urls and and brackets, parenthesis and hyphens, leaving only alphanumeric words
url_regex = r"([--:\w?@%&+~#=]*\.[a-z]{2,4}\/{0,2})((?:[?&](?:\w+)=(?:\w+))+|[--:\w?@%&+~#=]+)?"
special_character_regex = r"[\"'()[\]]"

news_df['body'] = news_df['body'].astype('str')
politics_df['body'] = politics_df['body'].astype('str')
politicaldiscussion_df['body'] = politicaldiscussion_df['body'].astype('str')

#remove urls, special characters, and replace hyphens with a space
news_df['clean'] = news_df['body'].apply(lambda text: text.strip().lower()).apply(lambda text: re.sub(url_regex, '', text)).apply(lambda text: re.sub(special_character_regex, '', text)).apply(lambda text: re.sub(r"-", ' ', text))
politics_df['clean'] = politics_df['body'].apply(lambda text: text.strip().lower()).apply(lambda text: re.sub(url_regex, '', text)).apply(lambda text: re.sub(special_character_regex, '', text)).apply(lambda text: re.sub(r"-", ' ', text))
politicaldiscussion_df['clean'] = politicaldiscussion_df['body'].apply(lambda text: text.strip().lower()).apply(lambda text: re.sub(url_regex, '', text)).apply(lambda text: re.sub(special_character_regex, '', text)).apply(lambda text: re.sub(r"-", ' ', text))

In [6]:
#tokenize
news_df['tokens'] = news_df['clean'].apply(lambda text: re.sub(r"[.,!?]"," ", text)).apply(lambda text: re.sub(r"[0-9]", " ", text)).apply(nltk.wordpunct_tokenize)
politics_df['tokens'] = politics_df['clean'].apply(lambda text: re.sub(r"[.,!?]"," ", text)).apply(lambda text: re.sub(r"[0-9]", " ", text)).apply(nltk.wordpunct_tokenize)
politicaldiscussion_df['tokens'] = politicaldiscussion_df['clean'].apply(lambda text: re.sub(r"[.,!?]"," ", text)).apply(lambda text: re.sub(r"[0-9]", " ", text)).apply(nltk.wordpunct_tokenize)

In [7]:
news_df = news_df.reset_index(drop=True)
politics_df = politics_df.reset_index(drop=True)
politicaldiscussion_df = politicaldiscussion_df.reset_index(drop=True)
print(len(news_df), "total comments from r/news")
print(len(politics_df), "total comments from r/politics")
print(len(politicaldiscussion_df), "total comments from r/PoliticalDiscussion")

17536 total comments from r/news
18857 total comments from r/politics
19174 total comments from r/PoliticalDiscussion


In [8]:
# build the test samples for evals
# since we trained using 80% of the total comments, we test on 20% of them.

import nltk

boundary = int(len(news_df)*0.8) #80/20 train/test split
news_df_test = news_df[boundary:]

boundary = int(len(politics_df)*0.8) #80/20 train/test split
politics_df_test = politics_df[boundary:]

boundary = int(len(politicaldiscussion_df)*0.8) #80/20 train/test split
politicaldiscussion_df_test = politicaldiscussion_df[boundary:]

news_test_vocab = nltk.lm.Vocabulary([word for sent in news_df_test['tokens'] for word in sent])
politics_test_vocab = nltk.lm.Vocabulary([word for sent in politics_df_test['tokens'] for word in sent])
politicaldiscussion_test_vocab = nltk.lm.Vocabulary([word for sent in politicaldiscussion_df_test['tokens'] for word in sent])


In [11]:
def preprocess(tokenizer, texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, len(tokenizer.word_index))

def generate_char(model, tokenizer, text, temperature=1):
    X_new = preprocess(tokenizer, [text])
    Y_pred = model.predict(X_new)[0, -1:, :]
    rescaled = tf.math.log(Y_pred)/temperature
    char_id = tf.random.categorical(rescaled, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def generate(model, tokenizer, text, n=50, temperature=1):
    for _ in range(n):
        text += generate_char(model, tokenizer, text, temperature)
    return text

In [18]:
import random
def random_char():
    return random.choice(list('abcdefghijklmnopqrstuvwxyz'))

random_char()

'n'

In [12]:
#make tokenizers for each of the models
news_all_text = ""
for comment in news_df['clean']:
    news_all_text += "<" + comment + ">"
    
politics_all_text = ""
for comment in politics_df['clean']:
    politics_all_text += "<" + comment + ">"
    
political_discussion_all_text = ""
for comment in politicaldiscussion_df['clean']:
    political_discussion_all_text += "<" + comment + ">"
    
news_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
news_tokenizer.fit_on_texts(news_all_text)

politics_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
politics_tokenizer.fit_on_texts(politics_all_text)

political_discussion_tokenizer = tf.keras.preprocessing.text.Tokenizer(char_level=True)
political_discussion_tokenizer.fit_on_texts(political_discussion_all_text)

In [10]:
# load models
news_model = keras.models.load_model('saved_models/NEWS_RNN_2_GRU_128_SOFTMAX_03_DROPOUT_EARLY_STOPPING', compile=False)
politics_model = keras.models.load_model('saved_models/POLITICS_RNN_2_GRU_128_SOFTMAX_03_DROPOUT_EARLY_STOPPING', compile=False)
political_discussion_model = keras.models.load_model('saved_models/POLDIS_RNN_2_GRU_128_SOFTMAX_03_DROPOUT_EARLY_STOPPING/', compile=False)

In [26]:
print("Fake sentence from r/news:\n", generate(news_model, news_tokenizer, random_char(), n=100, temperature=0.2),"\n")
print("Fake sentence from r/politics:\n", generate(politics_model, politics_tokenizer, random_char(), n=100, temperature=0.2),"\n")
print("Fake sentence from r/PoliticalDiscussion:\n", generate(political_discussion_model, political_discussion_tokenizer, random_char(), n=100, temperature=0.2),"\n")

Fake sentence from r/news:
 and the police are the rich and the cops were referring to the same than the police are the way that  

Fake sentence from r/politics:
 questions that is the people with a conservative with the worst commanity of the same and the america 

Fake sentence from r/PoliticalDiscussion:
 ke the market party in the country who want to be a control of the problem is that the result of the  



In [27]:
print("Perplexities of test vocabularies")
print("==============================")
print("r/news:", np.log(len(news_test_vocab)))
print("r/politics:", np.log(len(politics_test_vocab)))
print("r/PoliticalDiscussion:", np.log(len(politicaldiscussion_test_vocab)))

Perplexities of test vocabularies
r/news: 9.158309963881798
r/politics: 9.319374133000306
r/PoliticalDiscussion: 9.49363784611014


In [42]:
#losses from tensorflow logs
news_loss = 1.769
politics_loss = 1.49
political_discussion_loss = 1.71
print("Perplexities of models")
("==============================")
print("r/news:", np.e ** news_loss)
print("r/politics:", np.e ** politics_loss)
print("r/PoliticalDiscussion:", np.e ** political_discussion_loss)

Perplexities of models
r/news: 5.864985442469667
r/politics: 4.437095519003664
r/PoliticalDiscussion: 5.528961477624003
